In [4]:
import requests
from dotenv import load_dotenv
import os
import json
import base64   
import pandas as pd
import librosa
import numpy as np
import lyricsgenius
import langdetect
import re
import string


In [2]:
spotify_client_id = "dae4aecab9ff4dfeb227cb9ff279ebf3"
spotify_client_secret = "0120d846e45e44c4843a8f983eeeb4a2"

genius_client_id = "wZZ2RWc5mqp-5Pbz2W1rQJWE8LQ3pFBrb1Hw5_AOqgybq28mt7kjdjcG4zktCNbO"
genius_client_secret = "PefqBJHor_muDgTutGlaXXaxmzsI7TQCps9FQ3FwkUTT0WJIT3s0A5YA9mnFbfp_-CBhQF7b0omgE8kaM3dJ3w"
genius_access_token = "NUHHVpwnmbDYUYw8Padu0gQeHvYN4OsKYtE2MKNUpBUI6yR-xZXKY6S5NvCnFbiP"

lastfm_api_key = "97d5a64d5ba4a8bc580b752ceff3b87f"
lastfm_secret = "35175090bd61f6f16ac607bd26e5b1de"

Spotify API

In [55]:
def get_token():
    auth_string = spotify_client_id + ":" + spotify_client_secret
    auth_bytes = auth_string.encode("utf-8")
    auth_base64 = str(base64.b64encode(auth_bytes), "utf-8")

    url = "https://accounts.spotify.com/api/token"
    headers = {
        "Authorization" : "Basic " + auth_base64,
        "Content-Type": "application/x-www-form-urlencoded"
    }
    data = {"grant_type": "client_credentials"}
    result = requests.post(url,headers=headers, data=data)
    json_result= json.loads(result.content)
    token = json_result['access_token']
    return token

def get_auth_header(token):
    return {"Authorization": "Bearer " + token }


def get_song_details(song_id, auth_header):
    """Fetch song metadata using Spotify API."""
    url = f"https://api.spotify.com/v1/tracks/{song_id}"
    response = requests.get(url, headers=auth_header)
    return response.json()

def get_audio_features(song_id, auth_header):
    """Fetch audio features for a song."""
    url = f"https://api.spotify.com/v1/audio-features/{song_id}"
    response = requests.get(url, headers=auth_header)
    return response.json()

def get_audio_analysis(song_id, auth_header):
    """Fetch detailed audio analysis for a song."""
    url = f"https://api.spotify.com/v1/audio-analysis/{song_id}"
    response = requests.get(url, headers=auth_header)
    return response.json()


def get_artist_details(artist_id, auth_header):
    """Fetch artist metadata including popularity."""
    url = f"https://api.spotify.com/v1/artists/{artist_id}"
    response = requests.get(url, headers=auth_header)
    return response.json()

def download_song_preview(preview_url, song_id):
    if not preview_url:
        print(f"No preview available for song ID {song_id}")
        return None
    response = requests.get(preview_url)
    if response.status_code == 200:
        file_path = f"previews/{song_id}.mp3"
        with open(file_path, 'wb') as file:
            file.write(response.content)
        return file_path
    else:
        print(f"Failed to download preview for song ID {song_id}")
        return None

## the seeds must be their ids
def get_recommendations(token, seed_artists=None, seed_genres=None, seed_tracks=None):
    url = "https://api.spotify.com/v1/recommendations"
    headers = get_auth_header(token)
    params = {
        "limit": 20  # You can adjust this to fetch more or fewer tracks
    }
    if seed_artists:
        params["seed_artists"] = ','.join(seed_artists)
    if seed_genres:
        params["seed_genres"] = ','.join(seed_genres)
    if seed_tracks:
        params["seed_tracks"] = ','.join(seed_tracks)
    
    result = requests.get(url, headers=headers, params=params)
    
    try:
        result.raise_for_status()
        json_result = result.json()
        return json_result['tracks']
    except json.JSONDecodeError:
        print(f"Error decoding JSON response: {result.content}")
        return None
    except requests.exceptions.HTTPError as err:
        print(f"HTTP error: {err}")
        return None


Trending Artists and Songs

In [61]:
import requests
import json

# Today's Top Hits: 37i9dQZF1DXcBWIGoYBM5M
# Global Top 50: 37i9dQZEVXbMDoHDwVN2tF
# United States Top 50: 37i9dQZEVXbLRQDuF5jeBp
# Viral Hits: 37i9dQZF1DX2L0iB23Enbq

def get_top_artists_from_playlist(playlist_id):
    url = f"https://api.spotify.com/v1/playlists/{playlist_id}/tracks"
    headers = get_auth_header(get_token())
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        tracks_data = response.json()
        artists = []
        for item in tracks_data['items']:
            track_artists = item['track']['artists']
            for artist in track_artists:
                artists.append({
                    "name": artist['name'],
                    "id": artist['id']
                })
        return artists
    else:
        return "Failed to fetch data from Spotify."

def get_featured_playlists(country=None):
    url = "https://api.spotify.com/v1/browse/featured-playlists"
    headers = get_auth_header(get_token())
    params = {"country": country} if country else {}
    response = requests.get(url, headers=headers, params=params)
    return response.json() 

def get_top_tracks_from_playlists(playlist_id):
    url = f"https://api.spotify.com/v1/playlists/{playlist_id}/tracks"
    headers = get_auth_header(get_token())
    response = requests.get(url, headers=headers)
    return response.json() 

featured_playlists_global = get_featured_playlists()
featured_playlists_country = get_featured_playlists(country="US")  

playlist_id = "37i9dQZF1DXcBWIGoYBM5M"
top_tracks = get_top_tracks_from_playlists(playlist_id)
top_artists = get_top_artists_from_playlist(playlist_id)

# print(json.dumps(featured_playlists_global, indent=2))
# print(json.dumps(top_tracks, indent=2))
for artist in top_artists:
    print(artist['name'], artist['id'])

Tate McRae 45dkTj5sMRSjrmBSBeiHym
Dua Lipa 6M2wZ9GZgrQXHCFfjv46we
Ariana Grande 66CXWjxzNUsdJxJ2JdwvnR
Teddy Swims 33qOK5uJ8AR2xuQQAhHump
SZA 7tYKF4w9nC0nq9CsPZTHyP
Benson Boone 22wbnEMDvgVIAGdFeek6ET
Jack Harlow 2LIk90788K0zvyj2JJVwkJ
The Weeknd 1Xyo4u8uXC1ZmMpatF05PJ
JENNIE 250b0Wlc5Vk0CoUsaCY84M
Lily-Rose Depp 1pBLC0qVRTB5zVMuteQ9jJ
Drake 3TVXtAsR1Inumwj472S9r4
Sexyy Red 3DbwFQlvLxRSi2uX8mf81A
SZA 7tYKF4w9nC0nq9CsPZTHyP
Doja Cat 5cj0lLjcoR7YOSnhnX0Po5
Kali Uchis 1U1el3k54VvEUzo3ybLPlM
Peso Pluma 12GqGscKJx3aE4t07u7eVZ
21 Savage 1URnnhqYAYcrqrcwql10ft
Flo Milli 08PvCOlef4xdOr20jFSTPd
Beyoncé 6vWDO969PvNqNYHIOW5v0m
Noah Kahan 2RQXRUsr4IW1f3mKyKsy4B
Taylor Swift 06HL4z0CvFAxyc27GXpf02
Tyla 3SozjO3Lat463tQICI9LcE
Sabrina Carpenter 74KM79TiuVKeVCqs8QtB0B
Mitski 2uYWxilOVlUdk4oV9DvwqK
Good Neighbours 52N3KGrTWDRhdQJrgBTofE
Billie Eilish 6qqNVTkY8uBg9cP3Jd7DAH
Tate McRae 45dkTj5sMRSjrmBSBeiHym
Kenya Grace 7uMDnSZyUYNBPLhPMNuaM2
Doja Cat 5cj0lLjcoR7YOSnhnX0Po5
Olivia Rodrigo 1McMsnEElThX1kn

Librosa API

In [19]:
def extract_librosa_features(audio_path):
    y, sr = librosa.load(audio_path)

    tempo, beat_frames = librosa.beat.beat_track(y=y, sr=sr)
    beat_times = librosa.frames_to_time(beat_frames, sr=sr)

    features = {
        'mfcc': np.mean(librosa.feature.mfcc(y=y, sr=sr).T, axis=0),
        'chroma': np.mean(librosa.feature.chroma_stft(y=y, sr=sr).T, axis=0),
        'tempo' : tempo,
        'beat_times' : beat_times

    }
    return features

Cyanite AI API

In [23]:
def get_cyanite_insights(audio_path):
    # This is a placeholder function. You need to replace it with actual API calls to Cyanite.ai
    cyanite_api_key = "your_cyanite_api_key"
    headers = {"Authorization": f"Bearer {cyanite_api_key}"}
    # API call to Cyanite with the audio file
    insights = {}  # Parse the API response into a dictionary
    return insights

Genius API

In [33]:
def fetch_and_clean_lyrics(artist_name, song_name):
    # Normalize the song and artist names by removing common noise
    song_name_clean = re.sub(r"[\(].*?[\)]", "", song_name).split(' - ', 1)[0]
    artist_name_clean = re.sub(r"[\(].*?[\)]", "", artist_name)
    
    # Remove punctuation for better matching
    song_name_clean = song_name_clean.translate(str.maketrans('', '', string.punctuation))
    artist_name_clean = artist_name_clean.translate(str.maketrans('', '', string.punctuation))

    genius = lyricsgenius.Genius(genius_access_token, timeout=15)
    try:
        # Attempt to fetch the song lyrics
        song = genius.search_song(song_name_clean, artist_name_clean)
        if song and song.lyrics:
            lyrics = song.lyrics
            # Clean lyrics by removing sections like [Chorus] and any excess whitespace
            lyrics_cleaned = re.sub("[\(\[].*?[\)\]]", "", lyrics).replace("\n", " ").strip()
            return lyrics_cleaned.lower()  # Convert to lowercase
    except Exception as e:
        print(f"An error occurred while fetching lyrics for {song_name}: {e}")
    
    return "Lyrics not found"

Main Function for Song Info

In [27]:

def combine_song_data(song_ids):
    
    token = get_token()
    auth_header = get_auth_header(token)
    
    songs_data = []
    for song_id in song_ids:
        song_details = get_song_details(song_id, auth_header)
        audio_features = get_audio_features(song_id, auth_header)
        preview_url = song_details.get("preview_url")
        preview_path = download_song_preview(preview_url, song_id) if preview_url else None

        # Fetch artist details for the first artist
        artist_details = get_artist_details(song_details["artists"][0]["id"], auth_header) if song_details["artists"] else {}

        artist_name = song_details["artists"][0]["name"] if song_details["artists"] else ""
        song_name = song_details.get("name", "")
        featured_artists = ", ".join([artist['name'] for artist in song_details['artists'][1:]]) if len(song_details['artists']) > 1 else None
        lyrics = fetch_and_clean_lyrics(artist_name, song_name)
        language = langdetect.detect_langs(lyrics) if lyrics != "Lyrics not found" else None
        librosa_features = extract_librosa_features(preview_path) if preview_path else {}

        combined_data = {
            "song_id": song_id,
            "name": song_details.get("name"),
            "album": song_details["album"]["name"] if song_details.get("album") else None,
            "artist": song_details["artists"][0]["name"] if song_details["artists"] else None,
            "featured_artists": featured_artists,
            "genre": artist_details.get("genres")[0] if artist_details.get("genres") else None,
            "available_markets": len(song_details["available_markets"]) if song_details.get("available_markets") else 0,
            "release_date": song_details["album"]["release_date"] if song_details.get("album") else None,
            "artist_popularity": artist_details.get("popularity"),
            "duration_ms": song_details.get("duration_ms"),
            "danceability": audio_features.get("danceability"),
            "energy": audio_features.get("energy"),
            "instrumentalness": audio_features.get("instrumentalness"),
            "loudness": audio_features.get("loudness"),
            "speechiness": audio_features.get("speechiness"),
            "tempo": audio_features.get("tempo"),
            "time_signature": audio_features.get("time_signature"),
            "valence": audio_features.get("valence"),
            "preview_path": preview_path,
            "lyrics": lyrics,
            "language": language,
            **librosa_features
        }
        songs_data.append(combined_data)
    
    df = pd.DataFrame(songs_data)
    return df

# Example usage with dummy song IDs
song_ids = ["3n3Ppam7vgaVa1iaRUc9Lp", "0VjIjW4GlUZAMYd2vXMi3b", '7qiZfU4dY1lWllzX7mPBI3', '2XU0oxnq2qxCpomAAuJY8K', '7qEHsqek33rTcFNT9PFqLf',
            '2Fxmhks0bxGSBdJ92vM42m', '6v3KW9xbzN5yKLt9YKDYA2', '3KkXRkHbMCARz0aVfEt68P', '21jGcNKet2qwijlDFuPiPb', 
            '2b8fOow8UzyDFAE27YhOZM', '3PfIrDoz19wz7qK7tYeu62']  # Replace with actual song IDs
df = combine_song_data(song_ids)
df

Searching for "Mr Brightside" by The Killers...
Done.
Searching for "Blinding Lights" by The Weeknd...
Done.
Searching for "Shape of You" by Ed Sheeran...
Done.
Searching for "Dance Monkey" by Tones And I...
Done.
Searching for "Someone You Loved" by Lewis Capaldi...
Done.
Searching for "bad guy" by Billie Eilish...
Done.
Searching for "Señorita" by Shawn Mendes...
Done.
Searching for "Sunflower" by Post Malone...
Done.
Searching for "Circles" by Post Malone...
Done.
Searching for "Memories" by Maroon 5...
Done.
Searching for "Dont Start Now" by Dua Lipa...
Done.
                   song_id                                           name  \
0   3n3Ppam7vgaVa1iaRUc9Lp                                 Mr. Brightside   
1   0VjIjW4GlUZAMYd2vXMi3b                                Blinding Lights   
2   7qiZfU4dY1lWllzX7mPBI3                                   Shape of You   
3   2XU0oxnq2qxCpomAAuJY8K                                   Dance Monkey   
4   7qEHsqek33rTcFNT9PFqLf                   

In [17]:

def fetch_songwriter_producer_musicbrainz(artist_name, song_name):
    base_url = "http://musicbrainz.org/ws/2/recording/"
    params = {
        "query": f'recording:"{song_name}" AND artist:"{artist_name}"',
        "fmt": "json",
        "inc": "artist-rels work-rels"
    }
    response = requests.get(base_url, params=params)
    if response.status_code == 200:
        data = response.json()
        return data
    else:
        print("Failed to fetch data from MusicBrainz")
        return {}
    
def parse_songwriter_producer(data):
    songwriters = []
    producers = []
    works = []

    if 'recordings' in data:
        for recording in data['recordings']:
            if 'relations' in recording:
                # Fetching work relations
                for relation in recording['relations']:
                    if relation['type'] == 'work':
                        works.append(relation['work']['id'])
    
    # Now we have work IDs, we can fetch details for each work
    for work_id in works:
        work_data = fetch_work_details(work_id)
        if 'relations' in work_data:
            for relation in work_data['relations']:
                if relation['type'] == 'lyricist' or relation['type'] == 'composer':
                    songwriters.append(relation['artist']['name'])
                if relation['type'] == 'producer':
                    producers.append(relation['artist']['name'])
    
    return songwriters, producers

def fetch_work_details(work_id):
    base_url = f"http://musicbrainz.org/ws/2/work/{work_id}"
    params = {
        "fmt": "json",
        "inc": "artist-rels"
    }
    response = requests.get(base_url, params=params)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Failed to fetch work details for work ID {work_id}")
        return {}
    
artist_name = "Ed Sheeran"
song_name = "Shape of you"

data = fetch_songwriter_producer_musicbrainz(artist_name, song_name)
songwriters, producers = parse_songwriter_producer(data)
print(songwriters, producers)

[] []


In [18]:


def get_song_url_genius(access_token, artist_name, song_name):
    base_url = "https://api.genius.com/search"
    headers = {"Authorization": f"Bearer {access_token}"}
    params = {"q": f"{artist_name} {song_name}"}
    response = requests.get(base_url, headers=headers, params=params)
    if response.status_code == 200:
        data = response.json()
        hits = data['response']['hits']
        for hit in hits:
            if artist_name.lower() in hit['result']['primary_artist']['name'].lower():
                return hit['result']['url']
        return "URL not found."
    else:
        return "Failed to fetch data from Genius."

from bs4 import BeautifulSoup

def scrape_song_info_genius(song_url):
    response = requests.get(song_url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # Assuming 'song_body-lyrics' was a placeholder and the correct class names need to be found
        # Update the class names according to the current page structure
        song_info = soup.find('div', class_='correct-class-for-song-info')
        
        # Assuming the layout for 'Produced by' and 'Written by' sections can be found with updated selectors
        produced_by = 'Not found'
        written_by = 'Not found'
        
        if song_info:
            produced_by_section = song_info.find('div', text='Produced by')
            if produced_by_section:
                produced_by = produced_by_section.find_next_sibling('a').text
                
            written_by_section = song_info.find('div', text='Written by')
            if written_by_section:
                written_by = written_by_section.find_next_sibling('a').text
                
        return {
            'Produced by': produced_by,
            'Written by': written_by
        }
    else:
        return {
            'Produced by': 'Failed to fetch data',
            'Written by': 'Failed to fetch data'
        }


url = get_song_url_genius(genius_access_token, "Ed Sheeran", "Shape of you")
print(url)
print(scrape_song_info_genius(url))

https://genius.com/Ed-sheeran-shape-of-you-lyrics
{'Produced by': 'Not found', 'Written by': 'Not found'}


Last.FM API

In [9]:
base_url = 'http://ws.audioscrobbler.com/2.0/'

def lastfm_get(payload):
    headers = {'user-agent': 'DataCollectorBot'}
    payload['api_key'] = lastfm_api_key
    payload['format'] = 'json'
    response = requests.get(base_url, headers=headers, params=payload)
    return response.json()

# Function for each endpoint
def get_user_info(user):
    payload = {'method': 'user.getinfo', 'user': user}
    return lastfm_get(payload)

def get_loved_tracks(user):
    payload = {'method': 'user.getlovedtracks', 'user': user}
    return lastfm_get(payload)

def get_recent_tracks(user):
    payload = {'method': 'user.getrecenttracks', 'user': user}
    return lastfm_get(payload)

def get_top_albums(user):
    payload = {'method': 'user.gettopalbums', 'user': user}
    return lastfm_get(payload)

def get_top_artists(user):
    payload = {'method': 'user.gettopartists', 'user': user}
    return lastfm_get(payload)

def get_top_tags(user):
    payload = {'method': 'user.gettoptags', 'user': user}
    return lastfm_get(payload)

def get_top_tracks(user):
    payload = {'method': 'user.gettoptracks', 'user': user}
    return lastfm_get(payload)

def get_weekly_album_chart(user):
    payload = {'method': 'user.getweeklyalbumchart', 'user': user}
    return lastfm_get(payload)

def get_weekly_artist_chart(user):
    payload = {'method': 'user.getweeklyartistchart', 'user': user}
    return lastfm_get(payload)

def get_weekly_chart_list(user):
    payload = {'method': 'user.getweeklychartlist', 'user': user}
    return lastfm_get(payload)

def get_weekly_track_chart(user):
    payload = {'method': 'user.getweeklytrackchart', 'user': user}
    return lastfm_get(payload)


user_data = {
    'info': get_user_info('Bans77'),
    'loved_tracks': get_loved_tracks('Bans77'),
    'recent_tracks': get_recent_tracks('Bans77'),
    'top_albums': get_top_albums('Bans77'),
    'top_artists': get_top_artists('Bans77'),
    'top_tags': get_top_tags('Bans77'),
    'top_tracks': get_top_tracks('Bans77'),
    'weekly_album_chart': get_weekly_album_chart('Bans77'),
    'weekly_artist_chart': get_weekly_artist_chart('Bans77'),
    'weekly_chart_list': get_weekly_chart_list('Bans77'),
    'weekly_track_chart': get_weekly_track_chart('Bans77')
}



In [16]:
recent_tracks = user_data['recent_tracks']

# Print each track with its listen timestamp
for track in recent_tracks['recenttracks']['track']:
    track_name = track['name']
    artist_name = track['artist']['#text']
    # Check if the 'date' key exists
    if 'date' in track:
        listen_time = track['date']['#text']  # Human-readable timestamp
        listen_timestamp = track['date']['uts']  # Unix timestamp
        print(f"{artist_name} - {track_name} listened to at {listen_time} (timestamp: {listen_timestamp})")
    else:
        # If 'date' key does not exist, it might be currently playing
        print(f"{artist_name} - {track_name} is currently playing or missing timestamp data.")

cubbie - Endlos is currently playing or missing timestamp data.
Clairo - Bags listened to at 19 Feb 2024, 05:21 (timestamp: 1708320087)
Isabel LaRosa - Older listened to at 19 Feb 2024, 05:04 (timestamp: 1708319068)
Olivia Rodrigo - Can’t Catch Me Now - from The Hunger Games: The Ballad of Songbirds & Snakes listened to at 19 Feb 2024, 05:01 (timestamp: 1708318864)
Dilated Peoples - Who's Who listened to at 19 Feb 2024, 04:56 (timestamp: 1708318594)
Aero Chord - Surface listened to at 19 Feb 2024, 04:52 (timestamp: 1708318341)
Aero Chord - Break Them listened to at 19 Feb 2024, 04:48 (timestamp: 1708318083)
Bullet for My Valentine - Hand of Blood listened to at 19 Feb 2024, 04:44 (timestamp: 1708317876)
The Crystal Method - Born Too Slow listened to at 19 Feb 2024, 04:42 (timestamp: 1708317752)
The Prodigy - You'll Be Under My Wheels listened to at 19 Feb 2024, 04:38 (timestamp: 1708317515)
T.I. - 24's listened to at 19 Feb 2024, 04:34 (timestamp: 1708317255)
P$C - Do Ya Thing (feat. Y